<a href="https://colab.research.google.com/github/haiiro1/Capstone/blob/feature%2Fjuako/Preproceso_de_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp "/content/drive/My Drive/cv2_cuda/cv2.cpython-312-x86_64-linux-gnu.so" .

In [4]:
import cv2
cv2.__version__

'4.13.0-dev'

In [12]:
import os
import cv2
import glob
import numpy as np
from tqdm import tqdm

print(cv2.__version__)

class ImagePreprocessorCuda:
    def __init__(self, high_contrast=1.5, low_contrast=0.7):
        self.high_contrast = float(high_contrast)
        self.low_contrast  = float(low_contrast)

    @staticmethod
    def _contrast_preserve_color(gpu_bgr_u8, alpha: float):
        """
        Ajusta contraste preservando color modificando solo Y (YCrCb).
        GPU siempre que sea posible; si alguna etapa devuelve host, seguimos en host.
        Devuelve BGR en CPU (np.uint8).
        """
        def _to_host(x):
            return x.download() if hasattr(x, "download") else x

        try:
            # --- GPU: BGR -> YCrCb
            gpu_ycrcb = cv2.cuda.cvtColor(gpu_bgr_u8, cv2.COLOR_BGR2YCrCb)
            Y, Cr, Cb = cv2.cuda.split(gpu_ycrcb)

            # Contraste alrededor de 128: Y' = alpha*Y + 128*(1-alpha)
            gamma = 128.0 * (1.0 - float(alpha))
            Y2 = cv2.cuda.addWeighted(Y, float(alpha), Y, 0.0, gamma)

            # Merge (puede devolver GpuMat o ndarray según tu build)
            merged = cv2.cuda.merge((Y2, Cr, Cb))
            ycrcb_host = _to_host(merged)  # <-- seguro: si ya está en host, no intenta download

            # --- CPU: YCrCb -> BGR
            bgr_host = cv2.cvtColor(ycrcb_host, cv2.COLOR_YCrCb2BGR)
            return bgr_host

        except cv2.error:
            # Fallback completo en CPU
            bgr_host = _to_host(gpu_bgr_u8)
            ycrcb = cv2.cvtColor(bgr_host, cv2.COLOR_BGR2YCrCb)
            Y, Cr, Cb = cv2.split(ycrcb)
            Y2 = cv2.addWeighted(Y, float(alpha), Y, 0.0, 128.0*(1.0 - float(alpha)))
            ycrcb2 = cv2.merge((Y2, Cr, Cb))
            return cv2.cvtColor(ycrcb2, cv2.COLOR_YCrCb2BGR)

    @staticmethod
    def _rotate_gpu(gpu_img_8u, angle_deg):
        rows, cols = gpu_img_8u.size()
        center = (cols / 2.0, rows / 2.0)
        M = cv2.getRotationMatrix2D(center, angle_deg, 1.0)
        return cv2.cuda.warpAffine(
            gpu_img_8u, M, (cols, rows),
            flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101
        )

    def process_image(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        if img is None:
            print(f"⚠️ Skipping (no se pudo leer): {img_path}")
            return

        # Subir a GPU una vez
        gpu_8u = cv2.cuda_GpuMat()
        gpu_8u.upload(img)

        # Contrastes (devuelven np.uint8 BGR en CPU)
        high = self._contrast_preserve_color(gpu_8u, self.high_contrast)
        low  = self._contrast_preserve_color(gpu_8u, self.low_contrast)

        dirname, filename = os.path.split(img_path)
        name, ext = os.path.splitext(filename)
        cv2.imwrite(os.path.join(dirname, f"{name}_high{ext}"), high)
        cv2.imwrite(os.path.join(dirname, f"{name}_low{ext}"),  low)

        # Rotaciones sobre el original en GPU
        for suffix, angle in {"rot90": 90, "rot180": 180, "rot270": 270}.items():
            gpu_rot = self._rotate_gpu(gpu_8u, angle)
            cv2.imwrite(os.path.join(dirname, f"{name}_{suffix}{ext}"), gpu_rot.download())


    def process_directories(self, root_dir, extensions=(".jpg", ".png", ".jpeg", ".bmp", ".tif", ".tiff")):
        for ext in extensions:
            img_paths = glob.glob(os.path.join(root_dir, "**", f"*{ext}"), recursive=True)
            for img_path in tqdm(img_paths, desc=f"Procesando {ext} en {root_dir}"):
                self.process_image(img_path)

def setup_kaggle_dataset(dataset_name, dataset_dir="/content/dataset"):

    os.makedirs(dataset_dir, exist_ok=True)
    os.system(f"kaggle datasets download -d {dataset_name} -p {dataset_dir} --unzip")
    return dataset_dir


def main():
    kaggle_dataset = "jegodoy/tomato-dataset-for-capstone"
    root_dir = setup_kaggle_dataset(kaggle_dataset)

    processor = ImagePreprocessorCuda()

    print(f"✅ Dataset ready at: {root_dir}")

    sub_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

    if not sub_dirs:
        print("⚠️ No subdirectories found. Processing root directory only.")
        sub_dirs = [root_dir]

    for img_dir in sub_dirs:
        print(f"🚀 Processing directory: {img_dir}")
        processor.process_directories(img_dir)

    print("🎉 Preprocessing complete!")


if __name__ == "__main__":
    main()

4.13.0-dev
✅ Dataset ready at: /content/dataset
🚀 Processing directory: /content/dataset/tomato


Procesando .jpg en /content/dataset/tomato: 100%|██████████| 34219/34219 [02:54<00:00, 196.16it/s]
Procesando .png en /content/dataset/tomato: 0it [00:00, ?it/s]
Procesando .jpeg en /content/dataset/tomato: 0it [00:00, ?it/s]
Procesando .bmp en /content/dataset/tomato: 0it [00:00, ?it/s]
Procesando .tif en /content/dataset/tomato: 0it [00:00, ?it/s]
Procesando .tiff en /content/dataset/tomato: 0it [00:00, ?it/s]

🎉 Preprocessing complete!


In [13]:
import shutil
import os
from google.colab import files

folder_path = '/content/dataset/tomato'

zip_path = '/content/tomato.zip'
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


# files.download(zip_path)

'/content/tomato.zip'